In [1]:
import ROOT
from ROOT import TFile
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import copy
import pandas as pd 
import time
from os import listdir

Welcome to JupyROOT 6.15/01


In [2]:
class particle:
    def __init__(self, pid, fourvector, virtual_photon,ThetaPQ):
        #work in frame that is aligned with photon
        #the photon is aligned with z-direction and has 4-momentum q= (0,0,sqrt(Nu2+Q2),Nu) (note q2 = -Q2)        
        self.virtual_photon = virtual_photon
        Nu = virtual_photon.E()
        Q2 = -virtual_photon.M2()
        self.proton = ROOT.TLorentzVector()
        self.proton.SetPxPyPzE(0,0,0, 0.938)
        self.W = (virtual_photon + self.proton).M()
        incoming_e = ROOT.TLorentzVector()
        incoming_e.SetPxPyPzE(0,0,5.014,5.014)
        part1 = virtual_photon.Vect().Cross(incoming_e.Vect()).Unit()
        part2 = virtual_photon.Vect().Cross(fourvector.Vect()).Unit()
        sign  = np.sign(part1.Dot(fourvector.Vect()))
        self.PhiPQ = sign*np.arccos(part1.Dot(part2))
        #print ' Angle my way ', self.PhiPQ
        #print 'Hayk (radians) ' , PhiPQ*np.pi/180.0
        
        photon_pz = np.sqrt(Nu*Nu+Q2) #direction is positive by definition
        self.bcm = photon_pz/(Nu + 0.938)#photon-nucleon center-of-mass velocity 
        self.ycm = 0.5*np.log(( 1+self.bcm)/(1-self.bcm)) #photon-nucleon center-of-mass rapidity
        
        self.LorentzVector = fourvector #hadron four-vector
        self.PhiLab = self.LorentzVector.Phi()
        self.E = self.LorentzVector.E() #energy in lab frame
        self.vector = self.LorentzVector.Vect()
        self.Pt = self.vector.Perp(virtual_photon.Vect().Unit()) #pT with respect to photon direction
        self.Pl  = self.vector.Dot(virtual_photon.Vect().Unit()) #pL with respect to photon direction (in lab frame)
        self.y =  0.5*np.log( (self.E+self.Pl)/(self.E-self.Pl)) #rapidity in lab frame
        self.mT = np.sqrt(self.LorentzVector.M2() + self.Pt*self.Pt)
        self.y_star = self.y - self.ycm
        self.Pl_star = self.mT*np.sinh(self.y_star)
        self.Xf = 2.0*self.Pl_star/self.W 
        self.pid = pid
        self.Zh = self.E/Nu
        self.ThetaPQ = np.arctan(self.Pt/self.Pl)
        #print 'My COSPQ' , np.arctan(self.Pt/self.Pl)
        #print 'Hayk COSPQ', self.ThetaPQ
        
    def redefine(self, new_virtual_photon):
        #this methods recalculates variables that depend on virtual photon direction
        #this helps for event mixing. 
        incoming_e = ROOT.TLorentzVector()
        incoming_e.SetPxPyPzE(0,0,5.014,5.014)
        part1 = new_virtual_photon.Vect().Cross(incoming_e.Vect()).Unit()
        part2 = new_virtual_photon.Vect().Cross(self.LorentzVector.Vect()).Unit()
       
        sign  = np.sign(part1.Dot(self.LorentzVector.Vect()))
        self.PhiPQ = sign*np.arccos(part1.Dot(part2)) 
        self.Pt = self.LorentzVector.Vect().Perp(new_virtual_photon.Vect().Unit()) #pT with respect to photon direction
        self.Pl  = self.LorentzVector.Vect().Dot(new_virtual_photon.Vect().Unit()) #pL with respect to photon direction (in lab frame)
        self.y =  0.5*np.log( (self.E+self.Pl)/(self.E-self.Pl)) #rapidity in lab frame
        self.ThetaPQ = np.arctan(self.Pt/self.Pl)
        self.virtual_photon = new_virtual_photon
        
        return
        
        
        
    def print_properties(self):
        print 'Hello, let me introduce myself, i am particle pid = ' , self.pid, ' with index ', self.index, ', from event  #', self.ievt, ' Nu and W', self.Nu, ' ' , self.W
        print 'zh = ', self.Zh, ' phi_pq= ', self.PhiPQ, ' theta_pq=' , self.ThetaPQ, 'E = ', self.E, ' xf', self.Xf,'Pt ', self.Pt, ' Pl= ', self.Pl, ' rapidity=' ,  self.y
        print 'pid = ' , self.pid

In [4]:

def getDataframes(filename, Target=1,maxevents=1e9,tree_name='tree_data'):
    dphi = np.array([])
    ParticlesFromPrevious = []
    try:
        myfile = TFile.Open('%s'%filename,'READ')
    except:
        print("could not open file")
        
    myfile.Print()
    mytree = myfile.Get(tree_name)
    e_tree = myfile.Get('e_rec')
        
    print filename, ' has ', mytree.GetEntries(), ' entries'
    
    tupla_electron = {}
    tupla_electron = ['Q2']
    tupla_electron = ['Xb']
    tupla_electron = ['Nu']
    tupla_electron = ['W']

    
    tupla_hadron = {}
    tupla_hadron['pid'] = []
    tupla_hadron['xf'] = []
    tupla_hadron['xf_default'] = []
    tupla_hadron['z']  = []
    tupla_hadron['y']  = []
    tupla_hadron['ycm'] = []
    tupla_hadron['Q2'] = []
    tupla_hadron['Xb'] = []
    tupla_hadron['Nu'] = []
    tupla_hadron['W'] = []
    tupla_hadron['phi_pq'] = []
    tupla_hadron['theta_pq'] = []
    tupla_hadron['TargType'] = []
    tupla_hadron['mass'] = []
    tupla_hadron['phi_lab'] = []
    tupla_hadron['theta_lab'] = []
    
    
    start = time.time()

    for ievt  in range(mytree.GetEntries()):
        mytree.GetEntry(ievt)   
        e_tree.GetEntry(ievt)
        if mytree.W<2.05 or mytree.Q2<1.0: continue
        if ievt>maxevents: break
        if mytree.TargType!=Target: continue
        W = mytree.W
        Nu = mytree.Nu
        #get electron momentum:
        Pe = np.sqrt(e_tree.Pex*e_tree.Pex + e_tree.Pey*e_tree.Pey+ e_tree.Pez*e_tree.Pez)
        scattered_e = ROOT.TLorentzVector()
        scattered_e.SetPxPyPzE(e_tree.Pex, e_tree.Pey, e_tree.Pez, Pe)
        incoming_e = ROOT.TLorentzVector()
        incoming_e.SetPxPyPzE(0,0,5.014,5.014)
        virtual_photon  = incoming_e - scattered_e 
        virtual_photon_unitvector = virtual_photon.Vect().Unit()
        proton = ROOT.TLorentzVector()
        proton.SetPxPyPzE(0,0,0, 0.938)
        
        ##alternative calculation of virtual photon
        #scattered electron
        #E    = 5.014
        #Eprime = E-Nu
        #phi_e = -scattered_e.Vect().Phi()
        #alt_scattered_e = ROOT.TLorentzVector()
        #cos_thetae = 1-mytree.Q2/(2*E*Eprime)
        #sin_thetae = np.sqrt(1-cos_thetae*cos_thetae)
        #alt_scattered_e.SetPxPyPzE(Eprime*sin_thetae*np.cos(phi_e),
        #                       Eprime*sin_thetae*np.sin(phi_e),
        #                       Eprime*cos_thetae,
        #                       Eprime)
        #alt_virtual_photon  = incoming_e - alt_scattered_e
        
        #print 'Comparing normal and alternative virtual photons momentum'
        #print 'virtual ',virtual_photon.Px(), ' ', virtual_photon.Py(), ' ', virtual_photon.Pz(), ' ',virtual_photon.E()
        #print 'alt ' ,alt_virtual_photon.Px(), ' ', alt_virtual_photon.Py(), ' ',alt_virtual_photon.Pz(), ' ',alt_virtual_photon.E()
        #################################################
        
        
        
        #photon-nucleon center-of-mass energy and rapidity
        #the photon is aligned with z-direction and has 4-momentum q= (0,0,sqrt(Nu2+Q2),Nu) (note q2 = -Q2)
        
     
        #print 'W ',  W , 'electron W ', e_tree.W
        #print (virtual_photon + proton).M()
        #print ' Entering main loop over particles'
        for i in range(len(mytree.pid)):
            if abs(mytree.pid[i]) !=211: continue
            i_lv = ROOT.TLorentzVector()    
            i_lv.SetPxPyPzE(mytree.Px[i],mytree.Py[i],mytree.Pz[i],mytree.Zh[i]*Nu)
#            print i_lv.Phi(), '   ', mytree.Phi[i]*np.pi/180.0
            
            
            i_part = particle(mytree.pid[i], i_lv, virtual_photon, mytree.ThetaPQ[i] )
            #print i_part.LorentzVector.Phi()
            #print mytree.Phi[i]*np.pi/180.0
            #print i_part.LorentzVector.Theta(), '   ', mytree.Theta[i]*np.pi/180.0


            if i_part.Zh > 0.0: 
                tupla_trigger['TargType'].append(mytree.TargType)
                tupla_trigger['pid'].append(i_part.pid)
                tupla_trigger['xf'].append(i_part.Xf)
                tupla_trigger['xf_default'].append(mytree.Xf[i])
                tupla_trigger['z'].append(i_part.Zh)
                tupla_trigger['y'].append(i_part.y_star)
                tupla_trigger['ycm'].append(i_part.ycm)
                tupla_trigger['phi_pq'].append(i_part.PhiPQ)
                tupla_trigger['theta_pq'].append(i_part.ThetaPQ)
                tupla_trigger['mass'].append(i_part.LorentzVector.M())
                tupla_trigger['Q2'].append(mytree.Q2)
                tupla_trigger['Xb'].append(mytree.Xb)
                tupla_trigger['Nu'].append(mytree.Nu)
                tupla_trigger['W'].append(mytree.W)
                tupla_trigger['phi_lab'].append(i_part.LorentzVector.Phi())
                tupla_trigger['theta_lab'].append(mytree.Theta[i])#i_part.LorentzVector.Theta())
                #print 'mytree.Pt[i] ' , mytree.Pt[i], ' check: ' ,i_part.Vector.Perp(virtual_photon_unitvector)
                
        #print ' going for next event'    
        #print ' particles in event', len(particles
        ##end loop over events correlations    
    end = time.time()
    print 'Processed in',  end-start, 'seconds'
    

    df_trigger = pd.DataFrame(tupla_trigger)
    print 'Number of pairs with z>0.4, ' ,df_trigger.query('z>0.4').shape[0] 
    myfile.Close()
    return df_trigger

In [ ]:
#for a particle on-shell:
# E = mT cosh y 
# pz = mT sinh y
#mT2 = m2 + pt2
# rapidity in the ion-at-rest frame, with the photon direction is 
# y = 0.5 * ln( (E+pz)/(E-pz))
#rapidity of the target in the lab frame = 0 (at rest)
# y = y* + ycm
#bcm = az + bz/ a0 + b0 
#bcm = photon_z / Nu + m_{p}
#ycm = 0.5 * ln( 1+bcm/1-bcm)
#energy of the photon = Nu
#momentum of the photon (magnitude) = sqrt(Nu2 + Q2) 
#q2 = -Q2. (four-momentum transfer)

## Create dataframe df

In [5]:
df = {}

## Pb data

In [8]:
path = '/mnt/c/Users/marratia/Linux/Pb_data/'

Files = listdir(path) 
df['Pb'],df['Pb_mix'], df['Pb_trigger'] = [None,None,None]
df['D_Pb'],df['D_Pb_mix'], df['D_Pb_trigger'] = [None,None,None]

for name in Files:
    filename = path+name
    print filename
    trigger = getDataframes(filename,Target=2,maxevents=1e3)
    df['Pb_trigger'] = pd.concat([ df['Pb_trigger'], trigger])
    trigger = getDataframes(filename,Target=1,maxevents=1e3)
    df['D_Pb_trigger'] = pd.concat([ df['D_Pb_trigger'], trigger])

/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41762.root
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41762.root  has  200510  entries
Processed in 0.173324108124 seconds
Number of pairs with z>0.4,  18
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41762.root  has  200510  entries
Processed in 0.333590984344 seconds
Number of pairs with z>0.4,  99
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41763.root
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41763.root  has  190818  entries
Processed in 0.170422792435 seconds
Number of pairs with z>0.4,  14
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41763.root  has  190818  entries
Processed in 0.434112787247 seconds
Number of pairs with z>0.4,  113
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41765.root
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41765.root  has  23948  entries
Processed in 0.141988992691 seconds
Number of pairs with z>0.4,  22
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in log


0.324414014816 seconds
Number of pairs with z>0.4,  92
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41774.root
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41774.root  has  105985  entries
Processed in 0.140035867691 seconds
Number of pairs with z>0.4,  29
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41774.root  has  105985  entries
Processed in 0.318743944168 seconds
Number of pairs with z>0.4,  92
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41775.root
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41775.root  has  197591  entries
Processed in 0.150341033936 seconds
Number of pairs with z>0.4,  21
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41775.root  has  197591  entries
Processed in 0.420967102051 seconds
Number of pairs with z>0.4,  94
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41776.root
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41776.root  has  196901  entries
Processed in 0.145205020905 seconds
Number of pairs with z>

Processed in 0.200623035431 seconds
Number of pairs with z>0.4,  29
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41817.root  has  199309  entries
Processed in 0.486695051193 seconds
Number of pairs with z>0.4,  86
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41818.root
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41818.root  has  122750  entries
Processed in 0.21604013443 seconds
Number of pairs with z>0.4,  19
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41818.root  has  122750  entries
Processed in 0.482758045197 seconds
Number of pairs with z>0.4,  87
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41819.root
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41819.root  has  204557  entries
Processed in 0.200389146805 seconds
Number of pairs with z>0.4,  20
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41819.root  has  204557  entries
Processed in 0.485569953918 seconds
Number of pairs with z>0.4,  93
/mnt/c/Users/marratia/Linux/Pb_data/CFFTre

Processed in 0.407517910004 seconds
Number of pairs with z>0.4,  93
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41842.root
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41842.root  has  197052  entries
Processed in 0.209563016891 seconds
Number of pairs with z>0.4,  16
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41842.root  has  197052  entries
Processed in 0.450838088989 seconds
Number of pairs with z>0.4,  95
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41843.root
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41843.root  has  172544  entries
Processed in 0.187037944794 seconds
Number of pairs with z>0.4,  25
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41843.root  has  172544  entries
Processed in 0.421216964722 seconds
Number of pairs with z>0.4,  85
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41844.root
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41844.root  has  196999  entries
Processed in 0.183251142502 seconds
Number of 

Processed in 0.168082952499 seconds
Number of pairs with z>0.4,  19
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41877.root  has  197244  entries
Processed in 0.424378871918 seconds
Number of pairs with z>0.4,  112
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41878.root
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41878.root  has  191963  entries
Processed in 0.168005943298 seconds
Number of pairs with z>0.4,  14
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41878.root  has  191963  entries
Processed in 0.431663990021 seconds
Number of pairs with z>0.4,  111
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41879.root
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41879.root  has  191622  entries
Processed in 0.222327947617 seconds
Number of pairs with z>0.4,  13
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41879.root  has  191622  entries
Processed in 0.503242969513 seconds
Number of pairs with z>0.4,  105
/mnt/c/Users/marratia/Linux/Pb_data/CF

Number of pairs with z>0.4,  89
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41903.root
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41903.root  has  76682  entries
Processed in 0.182603120804 seconds
Number of pairs with z>0.4,  28
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41903.root  has  76682  entries
Processed in 0.416651010513 seconds
Number of pairs with z>0.4,  78
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41906.root
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41906.root  has  4493  entries
Processed in 0.178743839264 seconds
Number of pairs with z>0.4,  27
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41906.root  has  4493  entries
Processed in 0.434199094772 seconds
Number of pairs with z>0.4,  100
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41907.root
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41907.root  has  42061  entries
Processed in 0.166831970215 seconds
Number of pairs with z>0.4,  15
/mnt/c/Users/marrati

Processed in 0.178478002548 seconds
Number of pairs with z>0.4,  19
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41936.root  has  164276  entries
Processed in 0.447247028351 seconds
Number of pairs with z>0.4,  92
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41937.root
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41937.root  has  35894  entries
Processed in 0.1850669384 seconds
Number of pairs with z>0.4,  21
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41937.root  has  35894  entries
Processed in 0.44668507576 seconds
Number of pairs with z>0.4,  83
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41938.root
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41938.root  has  195941  entries
Processed in 0.179953813553 seconds
Number of pairs with z>0.4,  21
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41938.root  has  195941  entries
Processed in 0.459813117981 seconds
Number of pairs with z>0.4,  83
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb

Number of pairs with z>0.4,  85
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41968.root
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41968.root  has  159948  entries
Processed in 0.210999965668 seconds
Number of pairs with z>0.4,  14
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41968.root  has  159948  entries
Processed in 0.524405002594 seconds
Number of pairs with z>0.4,  97
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41969.root
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41969.root  has  192675  entries
Processed in 0.228838920593 seconds
Number of pairs with z>0.4,  21
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41969.root  has  192675  entries
Processed in 0.475204944611 seconds
Number of pairs with z>0.4,  100
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41970.root
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41970.root  has  167953  entries
Processed in 0.203027963638 seconds
Number of pairs with z>0.4,  18
/mnt/c/Users/

Processed in 0.217434167862 seconds
Number of pairs with z>0.4,  22
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41993.root  has  175435  entries
Processed in 0.445261955261 seconds
Number of pairs with z>0.4,  98
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41994.root
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41994.root  has  151678  entries
Processed in 0.170584201813 seconds
Number of pairs with z>0.4,  21
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41994.root  has  151678  entries
Processed in 0.461258888245 seconds
Number of pairs with z>0.4,  109
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41995.root
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41995.root  has  136953  entries
Processed in 0.186030864716 seconds
Number of pairs with z>0.4,  21
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41995.root  has  136953  entries
Processed in 0.47958612442 seconds
Number of pairs with z>0.4,  92
/mnt/c/Users/marratia/Linux/Pb_data/CFFTr

In [ ]:
df['Pb_mix'].keys()

## Fe Data

In [ ]:
path = '/mnt/c/Users/marratia/Linux/Fe_data/'

Files = listdir(path) 
df['Fe'],df['Fe_mix'], df['Fe_trigger'] = [None,None,None]
df['D_Fe'],df['D_Fe_mix'], df['D_Fe_trigger'] = [None,None,None]

for name in Files:
    filename = path+name
    pairs, pairs_mix, trigger = getDataframes(filename,Target=2)
    df['Fe'] = pd.concat([ df['Fe'], pairs])
    df['Fe_mix'] = pd.concat([ df['Fe_mix'], pairs_mix])
    df['Fe_trigger'] = pd.concat([ df['Fe_trigger'], trigger])
    pairs, pairs_mix, trigger = getDataframes(filename,Target=1)
    df['D_Fe'] = pd.concat([ df['D_Fe'], pairs])
    df['D_Fe_mix'] = pd.concat([ df['D_Fe_mix'], pairs_mix])
    df['D_Fe_trigger'] = pd.concat([ df['D_Fe_trigger'], trigger])

# C data

In [ ]:
path = '/mnt/c/Users/marratia/Linux/C_data/'
Files = listdir(path) 
df['C'],df['C_mix'], df['C_trigger'] = [None,None,None]
df['D_C'],df['D_C_mix'], df['D_C_trigger'] = [None,None,None]

for name in Files:
    filename = path+name
    pairs, pairs_mix, trigger = getDataframes(filename,Target=2)
    df['C'] = pd.concat([ df['C'], pairs])
    df['C_mix'] = pd.concat([ df['C_mix'], pairs_mix])
    df['C_trigger'] = pd.concat([ df['C_trigger'], trigger])
    pairs, pairs_mix, trigger = getDataframes(filename,Target=1)
    df['D_C'] = pd.concat([ df['D_C'], pairs])
    df['D_C_mix'] = pd.concat([ df['D_C_mix'], pairs_mix])
    df['D_C_trigger'] = pd.concat([ df['D_C_trigger'], trigger])

## MC

In [ ]:
path = '/mnt/c/Users/marratia/Linux/MC_C/'

Files = listdir(path) 
df['MC'],df['MC_mix'], df['MC_trigger'] = [None,None,None]
df['MC_true'],df['MC_true_mix'], df['MC_true_trigger'] = [None,None,None]

for name in Files:
    filename = path+name
    pairs, pairs_mix, trigger = getDataframes(filename,Target=2,tree_name='tree_accept')
    df['MC'] = pd.concat([ df['MC'], pairs])
    df['MC_mix'] = pd.concat([ df['MC_mix'], pairs_mix])
    df['MC_trigger'] = pd.concat([ df['MC_trigger'], trigger])

    pairs, pairs_mix, trigger = getDataframes(filename,Target=2,tree_name='tree_thrown')
    df['MC_true'] = pd.concat([ df['MC_true'], pairs])
    df['MC_true_mix'] = pd.concat([ df['MC_true_mix'], pairs_mix])
    df['MC_true_trigger'] = pd.concat([ df['MC_true_trigger'], trigger])



In [ ]:
import root_pandas
from root_pandas import read_root
from root_pandas import to_root 

## Save dataframes to ROOT files

In [ ]:
to_root(df['Pb'],'Pairs_Pb.root', key='Pb')
to_root(df['Pb_mix'],'Pairs_Pb.root', key='Pb_mix',mode='a')
to_root(df['Pb_trigger'],'Pairs_Pb.root', key='Pb_trigger', mode='a')
to_root(df['D_Pb'],'Pairs_Pb.root', key='D_Pb',mode='a')
to_root(df['D_Pb_mix'],'Pairs_Pb.root', key='D_Pb_mix',mode='a')
to_root(df['D_Pb_trigger'],'Pairs_Pb.root', key='D_Pb_trigger', mode='a')

In [ ]:
to_root(df['Fe'],'Pairs_Fe.root', key='Fe')
to_root(df['Fe_mix'],'Pairs_Fe.root', key='Fe_mix',mode='a')
to_root(df['Fe_trigger'],'Pairs_Fe.root', key='Fe_trigger', mode='a')
to_root(df['D_Fe'],'Pairs_Fe.root', key='D_Fe',mode='a')
to_root(df['D_Fe_mix'],'Pairs_Fe.root', key='D_Fe_mix',mode='a')
to_root(df['D_Fe_trigger'],'Pairs_Fe.root', key='D_Fe_trigger', mode='a')

In [ ]:
to_root(df['C'],'Pairs_C.root', key='C')
to_root(df['C_mix'],'Pairs_C.root', key='C_mix',mode='a')
to_root(df['C_trigger'],'Pairs_C.root', key='C_trigger', mode='a')
to_root(df['D_C'],'Pairs_C.root', key='D_C',mode='a')
to_root(df['D_C_mix'],'Pairs_C.root', key='D_C_mix',mode='a')
to_root(df['D_C_trigger'],'Pairs_C.root', key='D_C_trigger', mode='a')

In [ ]:
df['Pb_trigger'].hist(figsize=(12,12),bins=100)
plt.show()

In [ ]:
df['MC_true'].hist(figsize=(12,12),bins=100)
plt.show()

In [ ]:
df['Pb_mix'].hist(figsize=(12,12),bins=100)
plt.show()

## See distributions